## 仅供交叉验证 前馈神经网络（NNAR）-按趋势分类

In [1]:
import pickle
import numpy as np
import random
from utils import *
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
n_input = 11

读取数据

In [2]:
# gene_arr_path = r'../output/gene_editing/es_with_decay.array'
# transplant_arr_path = r'../output/transplant/es_with_decay.array'

# gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
# transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 17, 10)
Shape of the transplant array: (5141, 17, 10)


### 截断数据
2019年为无效数据

In [3]:
# gene_arr = gene_arr[:, :-1, :]
# transplant_arr = transplant_arr[:, :-1, :]

# print('Shape of the gene_editing array:',gene_arr.shape)
# print('Shape of the transplant array:',transplant_arr.shape)

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)


### 规范数据并获取5折交叉检验所需的训练集和验证集

In [4]:
# scaler, data = scale_data(transplant_arr, 'standard')

# # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
# X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2],transplant_arr[:, n_input, -1]
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 按趋势划分

In [2]:
def split_data_by_trend(data, targets):
    up_data = []
    down_data = []
    up_target = []
    down_target = []
    
    for i in range(len(data)):
        a, b = np.polyfit(range(len(data[i])), data[i, :, -2].reshape(-1), 1)
        if a > 0:
            up_data.append(data[i])
            up_target.append(targets[i])
        else:
            down_data.append(data[i])
            down_target.append(targets[i])
    return np.array(up_data), np.array(up_target), np.array(down_data), np.array(down_target)

### 构建模型

In [3]:
def root_mean_squared_error(y_true, y_pred):
        return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true), axis=-1)) 

def build_direct_dnn_model():
    model = keras.models.Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(5))
    
    optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer)
    return model

### 进行训练和评估
使用EarlyStopping和Checkpoint做训练停止方式

In [4]:
def cross_validation(X, y, y_cat, kfold, scaler):
    overall_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }

    annual_metrics = {
        'mae':[],
        'rmse':[],
        'ndcg':[],
        'mape':[],
        'r2':[],
        'pearson':[],
        'acc':[]
    }
    
    for train, test in kfold.split(X, y_cat):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        models = []
        
        # 按总量划分数据集
        X_train1, y_train1, X_train2, y_train2 = split_data_by_trend(X_train, y_train)
        train_xs = [X_train1, X_train2]
        train_ys = [y_train1, y_train2]
        
        X_test1, y_test1, X_test2, y_test2 = split_data_by_trend(X_test, y_test)
        test_xs = [X_test1, X_test2]
        test_ys = [y_test1, y_test2]
        i_s = [1, 2]
        
        # 训练
        for i in range(len(i_s)):
            model = build_direct_dnn_model()
            history = model.fit(train_xs[i], train_ys[i], epochs=100, batch_size=16, verbose=1, validation_data=(test_xs[i], test_ys[i]),
                           callbacks=[
                               EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto', restore_best_weights=True)
                           ])
            models.append(model)
        
        # 预测
        y_test = []
        y_pred = []
        for i in range(len(i_s)):
            y_test.append(test_ys[i])
            y_pred.append(models[i].predict(test_xs[i]).reshape(test_ys[i].shape))
        
        y_test = np.concatenate(y_test)
        y_pred = np.concatenate(y_pred)

        metrics = ['mae', 'rmse','ndcg', 'mape', 'r2', 'pearson', 'acc']
        for m in metrics:
            overall, annual = eval_model(m, y_test, y_pred, scaler)
            overall_metrics[m].append(overall)
            annual_metrics[m].append(annual)
    
    return overall_metrics, annual_metrics


In [5]:
def full_pipeline():
    gene_arr_path = r'../output/gene_editing/es_with_decay.array'
    transplant_arr_path = r'../output/transplant/es_with_decay.array'

    gene_arr = pickle.load(open(gene_arr_path, mode='rb'))
    transplant_arr = pickle.load(open(transplant_arr_path, mode='rb'))
    
    gene_arr = gene_arr[:, :-1, :]
    transplant_arr = transplant_arr[:, :-1, :]

    print('Shape of the gene_editing array:',gene_arr.shape)
    print('Shape of the transplant array:',transplant_arr.shape)
    
    metrics = {
        'gene':{
            'overall':{},
            'annual':{}
        },
        'transplant':{
            'overall':{},
            'annual':{}
        }
    }
    
    for name, dataset in zip(['gene', 'transplant'], [gene_arr, transplant_arr]):
        scaler, data = scale_data(dataset, 'standard')

        # 用预测第二年的类别变量作为分成Kfold的依据，不支持浮点数
        X, y, y_cat = data[:, :n_input, :], data[:, n_input:, -2], dataset[:, n_input, -1]
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        overall_metrics, annual_metrics = cross_validation(X, y, y_cat, kfold, scaler)
        
        for metric, value in overall_metrics.items():
            metrics[name]['overall'][metric] = np.mean(value)
        
        for metric, value in annual_metrics.items():
            metrics[name]['annual'][metric] = np.mean(np.array(value), axis=0)
    
    pickle.dump(metrics, open('mlp_metrics.dict', 'wb'))
    
    return metrics

In [6]:
metrics = full_pipeline()

Shape of the gene_editing array: (2643, 16, 10)
Shape of the transplant array: (5141, 16, 10)
Train on 988 samples, validate on 256 samples
Epoch 1/100
988/988 [==============================] - 1s 1ms/sample - loss: 1.2508 - val_loss: 1.1335
Epoch 2/100
988/988 [==============================] - 0s 191us/sample - loss: 1.0905 - val_loss: 1.0782
Epoch 3/100
988/988 [==============================] - 0s 186us/sample - loss: 1.0605 - val_loss: 1.0703
Epoch 4/100
988/988 [==============================] - 0s 188us/sample - loss: 1.0519 - val_loss: 1.0671
Epoch 5/100
988/988 [==============================] - 0s 189us/sample - loss: 1.0460 - val_loss: 1.0668
Epoch 6/100
988/988 [==============================] - 0s 197us/sample - loss: 1.0415 - val_loss: 1.0646
Epoch 7/100
988/988 [==============================] - 0s 187us/sample - loss: 1.0373 - val_loss: 1.0656
Epoch 8/100
988/988 [==============================] - 0s 187us/sample - loss: 1.0330 - val_loss: 1.0664
Epoch 9/100
988/988 [=

998/998 [==============================] - 0s 263us/sample - loss: 1.0435 - val_loss: 1.1104
Epoch 5/100
998/998 [==============================] - 0s 243us/sample - loss: 1.0377 - val_loss: 1.1031
Epoch 6/100
998/998 [==============================] - 0s 249us/sample - loss: 1.0345 - val_loss: 1.1010
Epoch 7/100
998/998 [==============================] - 0s 236us/sample - loss: 1.0296 - val_loss: 1.1017
Epoch 8/100
998/998 [==============================] - 0s 241us/sample - loss: 1.0259 - val_loss: 1.0998
Epoch 9/100
998/998 [==============================] - 0s 248us/sample - loss: 1.0231 - val_loss: 1.0997
Epoch 10/100
998/998 [==============================] - 0s 252us/sample - loss: 1.0190 - val_loss: 1.0992
Epoch 11/100
998/998 [==============================] - 0s 248us/sample - loss: 1.0165 - val_loss: 1.0963
Epoch 12/100
998/998 [==============================] - 0s 239us/sample - loss: 1.0143 - val_loss: 1.0970
Epoch 13/100
998/998 [==============================] - 0s 247us

995/995 [==============================] - 0s 245us/sample - loss: 1.0170 - val_loss: 1.0425
Epoch 17/100
995/995 [==============================] - 0s 281us/sample - loss: 1.0151 - val_loss: 1.0460
Epoch 18/100
995/995 [==============================] - 0s 258us/sample - loss: 1.0125 - val_loss: 1.0466
Epoch 19/100
995/995 [==============================] - 0s 271us/sample - loss: 1.0082 - val_loss: 1.0447
Epoch 20/100
995/995 [==============================] - 0s 252us/sample - loss: 1.0068 - val_loss: 1.0451
Epoch 21/100
995/995 [==============================] - 0s 260us/sample - loss: 1.0039 - val_loss: 1.0475
Epoch 22/100
995/995 [==============================] - 0s 238us/sample - loss: 1.0015 - val_loss: 1.0448
Epoch 23/100
995/995 [==============================] - 0s 243us/sample - loss: 1.0007 - val_loss: 1.0462
Epoch 24/100
995/995 [==============================] - 0s 278us/sample - loss: 0.9972 - val_loss: 1.0446
Epoch 25/100
995/995 [==============================] - 0s 

1004/1004 [==============================] - 0s 225us/sample - loss: 1.0337 - val_loss: 1.0255
Epoch 11/100
1004/1004 [==============================] - 0s 224us/sample - loss: 1.0306 - val_loss: 1.0264
Epoch 12/100
1004/1004 [==============================] - 0s 224us/sample - loss: 1.0271 - val_loss: 1.0234
Epoch 13/100
1004/1004 [==============================] - 0s 216us/sample - loss: 1.0243 - val_loss: 1.0259
Epoch 14/100
1004/1004 [==============================] - 0s 225us/sample - loss: 1.0218 - val_loss: 1.0266
Epoch 15/100
1004/1004 [==============================] - 0s 223us/sample - loss: 1.0193 - val_loss: 1.0246
Epoch 16/100
1004/1004 [==============================] - 0s 225us/sample - loss: 1.0162 - val_loss: 1.0228
Epoch 17/100
1004/1004 [==============================] - 0s 218us/sample - loss: 1.0138 - val_loss: 1.0217
Epoch 18/100
1004/1004 [==============================] - 0s 223us/sample - loss: 1.0119 - val_loss: 1.0212
Epoch 19/100
1004/1004 [=================

3531/3531 [==============================] - 1s 193us/sample - loss: 0.5812 - val_loss: 0.6360
Epoch 16/100
3531/3531 [==============================] - 1s 191us/sample - loss: 0.5776 - val_loss: 0.6321
Epoch 17/100
3531/3531 [==============================] - 1s 189us/sample - loss: 0.5755 - val_loss: 0.6344
Epoch 18/100
3531/3531 [==============================] - 1s 192us/sample - loss: 0.5728 - val_loss: 0.6324
Train on 580 samples, validate on 146 samples
Epoch 1/100
580/580 [==============================] - 1s 2ms/sample - loss: 0.5291 - val_loss: 0.5190
Epoch 2/100
580/580 [==============================] - 0s 248us/sample - loss: 0.4412 - val_loss: 0.4955
Epoch 3/100
580/580 [==============================] - 0s 248us/sample - loss: 0.4166 - val_loss: 0.4808
Epoch 4/100
580/580 [==============================] - 0s 258us/sample - loss: 0.4022 - val_loss: 0.4734
Epoch 5/100
580/580 [==============================] - 0s 267us/sample - loss: 0.3934 - val_loss: 0.4725
Epoch 6/100


Epoch 4/100
570/570 [==============================] - 0s 271us/sample - loss: 0.4112 - val_loss: 0.4257
Epoch 5/100
570/570 [==============================] - 0s 294us/sample - loss: 0.4016 - val_loss: 0.4245
Epoch 6/100
570/570 [==============================] - 0s 276us/sample - loss: 0.3927 - val_loss: 0.4248
Epoch 7/100
570/570 [==============================] - 0s 268us/sample - loss: 0.3877 - val_loss: 0.4260
Epoch 8/100
570/570 [==============================] - 0s 268us/sample - loss: 0.3824 - val_loss: 0.4267
Epoch 9/100
570/570 [==============================] - 0s 269us/sample - loss: 0.3783 - val_loss: 0.4270
Epoch 10/100
570/570 [==============================] - 0s 268us/sample - loss: 0.3749 - val_loss: 0.4272
Epoch 11/100
570/570 [==============================] - 0s 266us/sample - loss: 0.3713 - val_loss: 0.4269
Epoch 12/100
570/570 [==============================] - 0s 261us/sample - loss: 0.3668 - val_loss: 0.4265
Epoch 13/100
570/570 [==============================

3515/3515 [==============================] - 1s 216us/sample - loss: 0.5777 - val_loss: 0.6217
Epoch 17/100
3515/3515 [==============================] - 1s 222us/sample - loss: 0.5742 - val_loss: 0.6224
Epoch 18/100
3515/3515 [==============================] - 1s 209us/sample - loss: 0.5712 - val_loss: 0.6217
Epoch 19/100
3515/3515 [==============================] - 1s 211us/sample - loss: 0.5674 - val_loss: 0.6250
Epoch 20/100
3515/3515 [==============================] - 1s 222us/sample - loss: 0.5652 - val_loss: 0.6226
Epoch 21/100
3515/3515 [==============================] - 1s 220us/sample - loss: 0.5620 - val_loss: 0.6208
Epoch 22/100
3515/3515 [==============================] - 1s 228us/sample - loss: 0.5587 - val_loss: 0.6257
Epoch 23/100
3515/3515 [==============================] - 1s 235us/sample - loss: 0.5578 - val_loss: 0.6271
Train on 599 samples, validate on 127 samples
Epoch 1/100
599/599 [==============================] - 1s 2ms/sample - loss: 0.5322 - val_loss: 0.4658


In [6]:
metrics

{'gene': {'overall': {'mae': 0.7299490896746337,
   'rmse': 1.2918968704150449,
   'ndcg': 0.44835543717483367},
  'annual': {'mae': array([0.41098335, 0.57326106, 0.75468637, 0.88329084, 1.02752384]),
   'rmse': array([0.74612466, 0.97067574, 1.33166402, 1.48635163, 1.66849718]),
   'ndcg': array([0.46505046, 0.36096419, 0.14908492, 0.19261638, 0.15743147])}},
 'transplant': {'overall': {'mae': 0.7718043551135649,
   'rmse': 1.2752554128409044,
   'ndcg': 0.48122490028670706},
  'annual': {'mae': array([0.75218725, 0.77410503, 0.74279975, 0.7759474 , 0.81398235]),
   'rmse': array([1.30078556, 1.29846621, 1.21739863, 1.22831554, 1.31962679]),
   'ndcg': array([0.09773368, 0.07247887, 0.02053076, 0.10604713, 0.11611737])}}}

In [7]:
metrics

{'gene': {'overall': {'mae': 0.7297824076242991,
   'rmse': 1.288492394560198,
   'ndcg': 0.450425276583966,
   'mape': 4.379720797710389,
   'r2': 0.24096271379741033,
   'pearson': 0.5239841606169792,
   'acc': 0.33765443023983865},
  'annual': {'mae': array([0.41629491, 0.57294854, 0.75242516, 0.88069069, 1.02655274]),
   'rmse': array([0.75623629, 0.96455199, 1.31755807, 1.48652634, 1.66621578]),
   'ndcg': array([0.42948659, 0.36199947, 0.15691518, 0.21310819, 0.14353297]),
   'mape': array([3.27338697, 2.92294838, 3.34444352, 6.13713268, 6.22069244]),
   'r2': array([0.42536017, 0.28595323, 0.18519127, 0.05494907, 0.02053747]),
   'pearson': array([0.66756681, 0.55964269, 0.47305132, 0.33346943, 0.2686703 ]),
   'acc': array([0.57891309, 0.20543948, 0.25955221, 0.3371119 , 0.30725548])}},
 'transplant': {'overall': {'mae': 0.7741190799377897,
   'rmse': 1.2768263814311376,
   'ndcg': 0.521502225186101,
   'mape': 3.5752431968110856,
   'r2': 0.4178778432693182,
   'pearson': 0.65